In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
client_info = es.info()
print("Connected to Elasticsearch")
pprint(client_info.body)

Connected to Elasticsearch
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'nv4JrjX8SLeHDApMSiNUPA',
 'name': '98617b9485a1',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-09-02T22:04:47.310170297Z',
             'build_flavor': 'default',
             'build_hash': '253e8544a65ad44581194068936f2a5d57c2c051',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.1'}}


In [2]:
if es.ping():
    print("Connect to Elasticsearch!")
else:
    print("Connection failed!")
    

Connect to Elasticsearch!


In [5]:
import requests 

#fetch data from fakestoreAPI
response = "https://fakestoreapi.com/products"
response = requests.get(response)
products = response.json()

pprint(products)

[{'category': "men's clothing",
  'description': 'Your perfect pack for everyday use and walks in the forest. '
                 'Stash your laptop (up to 15 inches) in the padded sleeve, '
                 'your everyday',
  'id': 1,
  'image': 'https://fakestoreapi.com/img/81fPKd-2AYL._AC_SL1500_.jpg',
  'price': 109.95,
  'rating': {'count': 120, 'rate': 3.9},
  'title': 'Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops'},
 {'category': "men's clothing",
  'description': 'Slim-fitting style, contrast raglan long sleeve, '
                 'three-button henley placket, light weight & soft fabric for '
                 'breathable and comfortable wearing. And Solid stitched '
                 'shirts with round neck made for durability and a great fit '
                 'for casual fashion wear and diehard baseball fans. The '
                 'Henley style round neckline includes a three-button placket.',
  'id': 2,
  'image': 'https://fakestoreapi.com/img/71-3HjGNDUL._AC_SY879.

In [6]:
for product in products:
    doc = {
        "id": product["id"],
        "title": product["title"],
        "price": product["price"],
        "description": product["description"],
        "category": product["category"],
        "image": product["image"],  # Corrected field
        "rating": product["rating"]["rate"],  # Corrected field
        "review_count": product["rating"]["count"]
    }

    es.index(index="fakestore", document=doc)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1979268165.py, line 6)

In [10]:
# verify data in elasticsearch
res = es.search(index="fakestore", body={"query": {"match_all": {}}})

for hit in res["hits"]["hits"][:5]: #show first 5 records
    pprint(hit["_source"])

{'category': "men's clothing",
 'description': 'Your perfect pack for everyday use and walks in the forest. '
                'Stash your laptop (up to 15 inches) in the padded sleeve, '
                'your everyday',
 'id': 1,
 'image': 'https://fakestoreapi.com/img/81fPKd-2AYL._AC_SL1500_.jpg',
 'price': 109.95,
 'rating': 3.9,
 'review_count': 120,
 'title': 'Fjallraven - Foldsack No. 1 Backpack, Fits 15 Laptops'}
{'category': "men's clothing",
 'description': 'Slim-fitting style, contrast raglan long sleeve, three-button '
                'henley placket, light weight & soft fabric for breathable and '
                'comfortable wearing. And Solid stitched shirts with round '
                'neck made for durability and a great fit for casual fashion '
                'wear and diehard baseball fans. The Henley style round '
                'neckline includes a three-button placket.',
 'id': 2,
 'image': 'https://fakestoreapi.com/img/71-3HjGNDUL._AC_SY879._SX._UX._SY._UY_.jpg'